In [102]:
import numpy as np
import pandas as pd
import random


In [109]:
def update_revenue_profit(x,column_name):
    if(x["Sales Stage"] != "Closed"):
        return x[column_name] * np.random.uniform(96,100)        
    else:
        return x[column_name]
    
def update_end_date(x):
    if(x["Sales Stage"] != "Closed"):
        return x["Status End Date"]+pd.DateOffset(np.random.randint(5,50))
    else:
        return x["Status End Date"]


def create_dataset_by_status_percentage(data,status,percentage_next,percentage_open,status_previous):    
    #data = data.loc[data["Sales Stage"] != "Closed"] #Get rows where status is not Closed
    data = data.loc[data["Sales Stage"] == status_previous] #Get rows where status is not the previous status
    data.index = range(len(data.index))
    data["Sales Stage"] = "Closed" #First Set All rows to Closed
    rand_index = random.sample(range(0,data.shape[0]),int(percentage_next*float(data.shape[0]))) #randomly select rows not to mark closed
    rand_index_open = random.sample(range(0,data.shape[0]),int(percentage_open*float(data.shape[0]))) #randomly select rows not to mark closed
    data.loc[rand_index,"Sales Stage"] = status  #set the status of random indexes to "Status"
    data.loc[rand_index_open,"Sales Stage"] = status_previous  #set the status of random indexes to previous status as open opportunities
    data.loc[:,"Status Start Date"] = data.loc[:,"Status End Date"] #Set Start Date to End Date of Previous status    
    data.apply(lambda x:update_end_date(x),axis=1) #Update End Date to new date when status is not closed
    data.apply(lambda x:update_revenue_profit(x,"Revenue"),axis=1) #Update Revenue when status is not closed
    data.apply(lambda x:update_revenue_profit(x,"Net Revenue"),axis=1) #Update Net Revnue when status is not closed
    data.loc[rand_index_open,"Status End Date"] = None #Set Start Date to End Date of Previous status    
    data.index = range(len(data.index))
    return data;



In [139]:
df = pd.read_csv("data-raw.csv")
df["Status Start Date"] = pd.to_datetime(df["Status Start Date"])
df["Status End Date"] = pd.to_datetime(df["Status End Date"])

q = create_dataset_by_status_percentage(df,"Qualify",np.random.uniform(0.65,0.85),np.random.uniform(0.25,0.45),"Lead")
lead_open = q.loc[q["Sales Stage"] == "Lead"]["Name"]
df = df.loc[~df["Name"].isin(lead_open.values)]
df = df.append(q)

s = create_dataset_by_status_percentage(q,"Solution",np.random.uniform(0.55,0.75),np.random.uniform(0.15,0.20),"Qualify")
df = df.append(s)
p = create_dataset_by_status_percentage(s,"Proposal",np.random.uniform(0.60,0.87),np.random.uniform(0.18,0.28),"Solution")
df = df.append(p)
d = create_dataset_by_status_percentage(p,"Deal",np.random.uniform(0.78,0.95),np.random.uniform(0.15,0.25),"Proposal")
df = df.append(d)


C:\Users\ahmad22h\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ahmad22h\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [141]:
df.to_csv("opportunity_data_complete.csv")
